<a href="https://colab.research.google.com/github/itimes-digital/deep-learning-estudo/blob/main/serie_temporal_mult_previsores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
base = pd.read_csv('https://raw.githubusercontent.com/itimes-digital/deep-learning-estudo/main/dataset/petr4_treinamento.csv')
base.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-01-02,19.990000,20.209999,19.690001,19.690001,18.086271,30182600.0
1,2013-01-03,19.809999,20.400000,19.700001,20.400000,18.738441,30552600.0
2,2013-01-04,20.330000,20.620001,20.170000,20.430000,18.766001,36141000.0
3,2013-01-07,20.480000,20.670000,19.950001,20.080000,18.444506,28069600.0
4,2013-01-08,20.110001,20.230000,19.459999,19.500000,17.911745,29091300.0


In [4]:
base.dropna(inplace=True)

In [7]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1242 entries, 0 to 1244
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1242 non-null   object 
 1   Open       1242 non-null   float64
 2   High       1242 non-null   float64
 3   Low        1242 non-null   float64
 4   Close      1242 non-null   float64
 5   Adj Close  1242 non-null   float64
 6   Volume     1242 non-null   float64
dtypes: float64(6), object(1)
memory usage: 77.6+ KB


In [5]:
base_treinamento = base.iloc[:, 1:7].values

In [6]:
scaler = MinMaxScaler(feature_range=(0,1));
base_treinamento_normalizado = scaler.fit_transform(base_treinamento);

In [12]:
previsores = []
preco_real = []

for i in range(180, 1242):
  previsores.append(base_treinamento_normalizado[i - 180: i, 0:6]);
  preco_real.append(base_treinamento_normalizado[i, 0])

In [13]:
previsores, preco_real = np.array(previsores), np.array(preco_real)
previsores.shape

(1062, 180, 6)

In [15]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [ ]:
model = Sequential();
model.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores[1], 6)));
model.add(Dropout(0.3));

model.add(LSTM(units = 50, return_sequences = True));
model.add(Dropout(0.3));

model.add(LSTM(units = 50, return_sequences = True));
model.add(Dropout(0.3));

model.add(LSTM(units = 50));
model.add(Dropout(0.3));

model.add(Dense(units = 1 activation = 'sigmoid'));
model.compile(optimizer = 'adam', 
              loss = 'mean_squared_error', 
              metrics = ['mean_absolute_error'])

model.summary()


In [17]:
es = EarlyStopping(monitor = 'loss', 
                   min_delta = 1e-10, 
                   patience = 10,
                   verbose = 1)

rlr = ReduceLROnPlateau(monitor = 'loss',
                        factor = 0.2,
                        patience = 5,
                        verbose = 1)

mcp = ModelCheckpoint(filepath = 'pesos.h5',
                      monitor = 'loss',
                      save_best_only = True,
                      verbose = 1)

In [ ]:
history_ = model.fit(previsores, 
                    preco_real, 
                    epochs = 100, 
                    batch_size = 32, 
                    callbacks = [es, rlr, mcp])